<a href="https://colab.research.google.com/github/RyanChenJung/Airbnb-Renting-Optimizer/blob/szuyu%2Fchore/DataCleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import ast
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv('listings.csv')

In [2]:
# 1. Check data dimensions
print("Initial Data Shape:", df.shape)

# 2. Check data types and non-null counts
print("\n--- Data Info ---")
df.info()

# 3. Identify columns with the highest missing values
missing_values = df.isnull().sum().sort_values(ascending=False)
print("\n--- Top 20 Columns with Missing Values ---")
print(missing_values[missing_values > 0].head(20))

# 4. Inspect categorical distributions
print("\n--- Room Type Distribution ---")
print(df['room_type'].value_counts())

print("\n--- Top 10 Property Types ---")
print(df['property_type'].value_counts().head(10))

Initial Data Shape: (8660, 79)

--- Data Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8660 entries, 0 to 8659
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            8660 non-null   int64  
 1   listing_url                                   8660 non-null   object 
 2   scrape_id                                     8660 non-null   int64  
 3   last_scraped                                  8660 non-null   object 
 4   source                                        8660 non-null   object 
 5   name                                          8660 non-null   object 
 6   description                                   8498 non-null   object 
 7   neighborhood_overview                         4731 non-null   object 
 8   picture_url                                   8660 non-null   object 
 9   host_id      

In [3]:
# Drop irrelevant, highly missing, and target leakage columns
columns_to_drop = [
    'estimated_occupancy_l365d', 'estimated_revenue_l365d',
    'neighbourhood_group_cleansed', 'calendar_updated',
    'listing_url', 'picture_url', 'host_url', 'host_thumbnail_url', 'host_picture_url',
    'scrape_id', 'last_scraped', 'source', 'calendar_last_scraped',
    'name', 'host_name', 'host_id',
    'description', 'neighborhood_overview', 'host_about',
    'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
    'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm',
    'availability_60', 'availability_90', 'availability_eoy',
    'calculated_host_listings_count_entire_homes',
    'calculated_host_listings_count_private_rooms',
    'calculated_host_listings_count_shared_rooms',
    'host_since', 'first_review', 'last_review', 'license', 'host_verifications',
    'neighbourhood', 'host_location', 'host_neighbourhood',
    'host_response_time', 'host_response_rate', 'host_acceptance_rate',
    'bathrooms_text'
]

df_clean = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

# Clean 'price' column (convert string to float)
df_clean['price'] = df_clean['price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
df_clean['price'] = pd.to_numeric(df_clean['price'], errors='coerce')

print("Data Shape after dropping columns:", df_clean.shape)

Data Shape after dropping columns: (8660, 36)


In [4]:
# Impute core numerical features with median
core_num_features = ['price', 'beds', 'bathrooms', 'bedrooms']
for col in core_num_features:
    df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Impute review scores and host metrics with median
review_cols = [col for col in df_clean.columns if 'review_scores' in col] + ['reviews_per_month', 'host_listings_count', 'host_total_listings_count']
for col in review_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna(df_clean[col].median())

# Verify missing values for numerical columns are handled
print("Missing values in core numerical features:", df_clean[core_num_features].isnull().sum().sum())

Missing values in core numerical features: 0


In [5]:
# Convert boolean strings (t/f) to binary (1/0)
binary_cols = ['host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability']
for col in binary_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].map({'t': 1, 'f': 0}).fillna(0)

# Consolidate 'property_type' to top 5 categories, label rest as 'Other'
top_5_properties = df_clean['property_type'].value_counts().nlargest(5).index
df_clean['property_type_clean'] = df_clean['property_type'].apply(lambda x: x if x in top_5_properties else 'Other')

# Apply One-Hot Encoding
df_encoded = pd.get_dummies(df_clean[['room_type', 'property_type_clean']], drop_first=True)
df_clean = pd.concat([df_clean, df_encoded], axis=1)

# Drop original categorical columns
df_clean = df_clean.drop(columns=['room_type', 'property_type', 'property_type_clean'])

print("Data Shape after encoding:", df_clean.shape)

Data Shape after encoding: (8660, 42)


In [6]:
# Safely parse stringified lists
def parse_amenities(amenity_string):
    try:
        return ast.literal_eval(amenity_string)
    except:
        return []

df_clean['amenities_list'] = df_clean['amenities'].fillna('[]').apply(parse_amenities)
df_clean['amenities_joined'] = df_clean['amenities_list'].apply(lambda x: "|".join(x))

def split_by_pipe(text):
    return text.split('|') if text else []

# Extract top 100 amenities
cv = CountVectorizer(tokenizer=split_by_pipe, max_features=100, token_pattern=None)
amenities_matrix = cv.fit_transform(df_clean['amenities_joined']).toarray()

# Dimensionality Reduction (PCA to 5 components)
pca_amenities = PCA(n_components=5, random_state=42)
amenities_components = pca_amenities.fit_transform(amenities_matrix)

# Append PCA components to the dataframe
for i in range(5):
    df_clean[f'Amenity_PCA_{i+1}'] = amenities_components[:, i]

# Drop intermediate amenity columns
df_clean = df_clean.drop(columns=['amenities', 'amenities_list', 'amenities_joined'])

print("Explained Variance Ratio of Amenity PCA Components:", pca_amenities.explained_variance_ratio_)

Explained Variance Ratio of Amenity PCA Components: [0.17712148 0.07737651 0.04835466 0.03966401 0.0294857 ]


In [8]:
import pandas as pd

# Extract the top 100 amenity names identified by CountVectorizer
feature_names = cv.get_feature_names_out()

# Create a DataFrame to map the PCA weights to the feature names
pca_components_df = pd.DataFrame(
    pca_amenities.components_,
    columns=feature_names,
    index=[f'Amenity_PCA_{i+1}' for i in range(5)]
)

# Print the top 10 heavily weighted amenities for each Principal Component
for i in range(5):
    pca_name = f'Amenity_PCA_{i+1}'
    print(f"\n🌟 Top Features for {pca_name}:")

    # Sort weights in descending order and get the top 10
    top_10_features = pca_components_df.loc[pca_name].sort_values(ascending=False).head(10)

    for feature, weight in top_10_features.items():
        # Display features that have a meaningful impact (weight > 0.05)
        if weight > 0.05:
            print(f"   - {feature} (Weight: {weight:.2f})")
    print("-" * 40)


🌟 Top Features for Amenity_PCA_1:
   - freezer (Weight: 0.22)
   - wine glasses (Weight: 0.22)
   - toaster (Weight: 0.21)
   - coffee (Weight: 0.21)
   - dining table (Weight: 0.21)
   - cleaning products (Weight: 0.21)
   - extra pillows and blankets (Weight: 0.19)
   - hot water kettle (Weight: 0.19)
   - baking sheet (Weight: 0.19)
   - shower gel (Weight: 0.18)
----------------------------------------

🌟 Top Features for Amenity_PCA_2:
   - oven (Weight: 0.30)
   - coffee maker (Weight: 0.30)
   - heating (Weight: 0.28)
   - refrigerator (Weight: 0.22)
   - shampoo (Weight: 0.21)
   - dishwasher (Weight: 0.19)
   - stove (Weight: 0.19)
   - long term stays allowed (Weight: 0.17)
   - elevator (Weight: 0.17)
   - essentials (Weight: 0.16)
----------------------------------------

🌟 Top Features for Amenity_PCA_3:
   - washer (Weight: 0.34)
   - stove (Weight: 0.33)
   - dryer (Weight: 0.27)
   - clothing storage (Weight: 0.20)
   - air conditioning (Weight: 0.20)
   - dishes and s

In [7]:
# Validate final dataset readiness
final_missing = df_clean.isnull().sum().max()
print("Final Data Shape:", df_clean.shape)
print("Maximum missing values in any column:", final_missing)

if final_missing == 0:
    print("Dataset is clean and ready for clustering.")

# Export the master feature set for the team
df_clean.to_csv('chicago_airbnb_cleaned.csv', index=False)

Final Data Shape: (8660, 46)
Maximum missing values in any column: 0
Dataset is clean and ready for clustering.
